In [1]:
from sklearn.neighbors import KNeighborsClassifier

from components.config import Config
from components.predictor import SklearnPredictor

sklearn_config = Config(model=KNeighborsClassifier(),
                model_filename_path = "../models",
                data_path='../data',
                dataset_split_path= "../split",
                dataset_name="mtat-20")
sklearn_predictor = SklearnPredictor(sklearn_config, model_filename="model.bin")

Cannot use cuda for model, defaulting to cpu


In [2]:
import numpy as np

test_files = np.load("../split/mtat-20/test.npy", allow_pickle=True)
tags = np.load("../split/mtat-20/tags.npy", allow_pickle=True)
binary = {row[0]: row[1:] for row in np.load("../split/mtat-20/binary.npy", allow_pickle=True)}

In [3]:
from components.preprocessor import OpenL3PreProcessor

preprocessor = OpenL3PreProcessor(input_path="../data/mtat/mp3", output_path="../data/mtat/emb", suffix="npy", sr=16000)
preprocessor.run(test_files[:, 1])

100%|██████████| 1972/1972 [00:07<00:00, 280.49it/s]


In [4]:
import pathlib
import os
from tqdm import tqdm

X = []
Y_true = []
for idx, file in tqdm(test_files):
    file = os.path.join("../data/mtat/emb/", pathlib.Path(file).with_suffix(f".npy"))
    X.append(np.load(file, allow_pickle=True))
    Y_true.append(binary[idx])
Y_true = np.array(Y_true)

100%|██████████| 1972/1972 [00:11<00:00, 166.37it/s]


In [5]:
Y_pred = []
for x in tqdm(X[:50]):
    Y_pred.append(sklearn_predictor.predict_data_prob(x))
Y_pred = np.array(Y_pred)

100%|██████████| 50/50 [00:22<00:00,  2.18it/s]


In [12]:
Y_pred = Y_pred.reshape(Y_pred.shape[0], -1)

In [13]:
Y_pred.shape

(50, 20)

In [14]:
Y_true = Y_true[:50]

In [16]:
from sklearn.metrics import classification_report
from components.common import Statistics

stats = Statistics(0, Y_pred, Y_true)
est_bin_array = np.where(stats.est_array >= 0.5, 1, 0)
print(classification_report(stats.gt_array, est_bin_array, target_names=list(sklearn_predictor.tags), zero_division=1))

[2023-05-01 18:13:39] Loss/Valid: 0.0000
[2023-05-01 18:13:39] F1 Score: 0.6258
[2023-05-01 18:13:39] AUC/ROC: 0.9637
[2023-05-01 18:13:39] AUC/PR: 0.8290
              precision    recall  f1-score   support

      guitar       0.71      0.56      0.63         9
   classical       0.92      0.71      0.80        17
        slow       0.67      0.36      0.47        11
      techno       1.00      1.00      1.00         7
     strings       0.56      0.50      0.53        10
       drums       1.00      0.71      0.83         7
  electronic       1.00      0.50      0.67         6
        rock       1.00      0.75      0.86         4
        fast       1.00      0.44      0.62         9
       piano       1.00      0.57      0.73         7
     ambient       0.67      0.67      0.67         6
        beat       1.00      0.71      0.83         7
      violin       0.71      1.00      0.83         5
       vocal       1.00      0.25      0.40         4
       synth       1.00      0.25 